FASTAPI

In [12]:
# !pip install langchain
# !pip install pymongo
# !pip install fastapi nest-asyncio pyngrok uvicorn

In [13]:
import os
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from datetime import datetime
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.memory import MongoDBChatMessageHistory
import urllib

In [14]:
import pickle

In [15]:
# Get the current working directory
cwd = os.getcwd()
Embedding_store_path = cwd

In [16]:
def load_embeddings(sotre_name, path):
    with open(f"{path}/faiss_{sotre_name}.pkl", "rb") as f:
        VectorStore = pickle.load(f)
    return VectorStore

In [17]:
db_instructEmbedd = load_embeddings(sotre_name='instructEmbeddings',
                                     path=Embedding_store_path)

retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 5})

In [18]:
import os
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "insert token"
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
llm=HuggingFaceHub(cache=None, verbose=True, callbacks=None, callback_manager=BaseCallbackManager([
        StreamingStdOutCallbackHandler()
    ]), repo_id='google/flan-t5-large', model_kwargs={'temperature': 0.05, 'max_length': 700})

In [19]:
memory = VectorStoreRetrieverMemory(retriever=retriever)
DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI about a university named FAST. The AI is talkative and provides lots of specific details from its context. The AI is not a student of the University of Lahore. If the AI does not know the answer to a question, it truthfully says it does not know.

 Relevant pieces of previous conversation:
 {history}

 (You do not need to use these pieces of information if not relevant)

 Current conversation:
 Human: {input}
 AI:"""



PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=DEFAULT_TEMPLATE
)
chatbot = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True
)
#chatbot.predict(input="Hi,what are the admission requirements?")

In [20]:
def fastbot (id, query):
  connection_string = "insert connection string"
  message_history = MongoDBChatMessageHistory(connection_string=connection_string, session_id="{id}")
  response = chatbot(
        {"history": message_history.messages, "input": query}
    )

#chatbot.predict(input="Hi,how many camouses does FAST have?")

  message_history.add_user_message(response["input"])
  message_history.add_ai_message(response["response"])

  return response["response"]

In [21]:
#res= bot("t5", "Hi, tell me about FAST's AI department ")


In [22]:
#res

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
from fastapi import Body

class Query(BaseModel):
  id: str
  query: str = Body()

app = FastAPI()

@app.get("/welcome")
def welcome():
    return {"Welcome to FAST's chatbot"}

@app.post("/bot/query")
def bot(id:str, query: str, session : Query):

    connection_string = "mongodb+srv://rubab05fatima:" + urllib.parse.quote("chatbot12345") + "@cluster0.btkngmn.mongodb.net/?retryWrites=true&w=majority"
    message_history = MongoDBChatMessageHistory(connection_string=connection_string, session_id=id)
    response = chatbot(
        {"history": message_history.messages, "input": query}
    )

#chatbot.predict(input="Hi,how many camouses does FAST have?")
    res= response["response"]
    message_history.add_user_message(response["input"])
    message_history.add_ai_message(response["response"])
    return {"response":res }

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000) 

Public URL: https://8223-154-192-46-17.ngrok.io


INFO:     Started server process [22220]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62402 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62402 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:62402 - "GET /openapi.json HTTP/1.1" 200 OK


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI about a university named FAST. The AI is talkative and provides lots of specific details from its context. The AI is not a student of the University of Lahore. If the AI does not know the answer to a question, it truthfully says it does not know.

 Relevant pieces of previous conversation:
 More Details

Bachelor of Science (Software Engineering)
Career Opportunities

Your career prospects will be excellent: You may become software quality assurance engineer, systems programmer, embedded software engineer, software architect, Chief Technology Officer, Chief Information Officer, User Interface designer, Information Systems manager or computer graphic designer etc.
Award of Degr


> Finished chain.
INFO:     127.0.0.1:62407 - "POST /bot/query?id=i200774&query=Hi%2C%20What%20is%20FAST%27s%20vision HTTP/1.1" 200 OK
